In [35]:
# 데이터셋 로더 설정

from torchvision import transforms, datasets
from torch.utils.data import DataLoader

transform = transforms.Compose([
        transforms.Resize((200,200)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
train_dataset = datasets.ImageFolder(root='/content/drive/MyDrive/image_classification_dataset/train', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)


In [41]:
import torch
import torch.nn as nn

# resnet50 정의

class ResidualBlock(nn.Module):
    expansion = 4

    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels * self.expansion, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels * self.expansion)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels * self.expansion:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * self.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * self.expansion)
            )

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = torch.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = torch.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=1000):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self.make_layer(block, 64, layers[0], stride=1)
        self.layer2 = self.make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self.make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self.make_layer(block, 512, layers[3], stride=2)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def make_layer(self, block, out_channels, blocks, stride):
        layers = []
        layers.append(block(self.in_channels, out_channels, stride))
        self.in_channels = out_channels * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

def resnet50(num_classes=1000):
    return ResNet(ResidualBlock, [3, 4, 6, 3], num_classes)




# 모델 정의
from torchvision.models import resnet50

class CustomResNet(nn.Module):
    def __init__(self, num_classes=6):
        super(CustomResNet, self).__init__()
        self.resnet = resnet50(pretrained=True)
        in_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(in_features, num_classes)

    def forward(self, x):
        return self.resnet(x)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
animalCF = CustomResNet(num_classes=6).to(device)
#animalCF = resnet50(num_classes=6).to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [42]:
# 모델 훈련

import torch.optim as optim
from torch.optim import lr_scheduler

animalCF.train()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(animalCF.parameters(), lr = 0.001, momentum=0.9)
#scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
epochs = 10

for ep in range(epochs):
  running_loss = 0.0
  batch_count = 0
  for imgs, lbls in train_loader:
    batch_count += 1
    imgs = imgs.to(device)
    lbls = lbls.to(device)

    optimizer.zero_grad()
    out = animalCF(imgs)
    loss = criterion(out, lbls)
    loss.backward()
    optimizer.step()

    #print(f"{batch_count} batch loss: {loss.item()}")
    running_loss += loss.item()

  #scheduler.step()
  print(f"Epoch {ep+1}, Loss: {running_loss/len(train_loader):.4f}")

torch.save(animalCF.state_dict(), './model_weights.pth')


Epoch 1, Loss: 1.4746
Epoch 2, Loss: 0.5990
Epoch 3, Loss: 0.2771
Epoch 4, Loss: 0.1646
Epoch 5, Loss: 0.1143
Epoch 6, Loss: 0.0869
Epoch 7, Loss: 0.0659
Epoch 8, Loss: 0.0516
Epoch 9, Loss: 0.0379
Epoch 10, Loss: 0.0308


In [44]:
# 모델 테스트

test_dataset = datasets.ImageFolder(root='/content/drive/MyDrive/image_classification_dataset/test', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

animalCF = CustomResNet(num_classes=6).to(device)
animalCF.load_state_dict(torch.load('./model_weights.pth'))
animalCF.eval()

total = 0
correct = 0
with torch.no_grad():
    for imgs, lbls in test_loader:
        imgs = imgs.to(device)
        lbls = lbls.to(device)
        out = animalCF(imgs)
        _, predicted = torch.max(out, 1)
        total += lbls.size(0)
        correct += (predicted == lbls).sum().item()

print(f"Accuracy : {100*correct/total}%")

Accuracy : 97.83333333333333%
